In [380]:
# Exploring sympy as a way of building up the model
from sympy import *

In [381]:
# TOPOLOGY:
N_f=Symbol("N_f") # global number of faces
N_c=Symbol("N_c") # global number of cells

f,c,fi=symbols('f c fi') # ,cls=Idx) # Generic face and cell indices,  side index of a cell
fw=Wild('f')
cw=Wild('c')

# Kleptsova uses the sparse s[f,c] connection between edges and cell
# s[f,c] is +1 if face f is adjacent to cell c and f's normal is out of
# c.  -1 if the normal is into c.  0 if f is not adjacent to c.
#sfc=IndexedBase("s")


# I'd rather use a more direct description:
Nf_max=Symbol('N_f,max') # maximum number of sides for a cell
Nf_c=IndexedBase('Nf') # number of sides for cell c
F=MatrixSymbol('F',N_c,Nf_max) # F[c,i] the i'th face adjacent to cell c 
S=MatrixSymbol('S',N_c,Nf_max) # S[c,i] sign of the normal of F[c,i] w.r.t. c's outward normal

# Question: what is the right convention here for boundary edges?
c1=IndexedBase('c1') # 'upstream' cell for edge f
c2=IndexedBase('c2') # 'downstream' cell for edge f

# constant GEOMETRY:
A_c=IndexedBase("A") # cell area
lf=IndexedBase("l") # length of face f
b=IndexedBase('b') # cell bed elevation

# variable GEOMETRY:
h_fn=IndexedBase("hf^n") # edge depth
h_cn=IndexedBase('hc^n') # water column depth
eta_n=IndexedBase(r'\eta^n')
eta_np1=IndexedBase(r'\eta^{n+1}')

# DYNAMIC
u_fn=IndexedBase("u^n")       # edge normal velocity
u_fnp1=IndexedBase("u^{n+1}") #
a_fn=IndexedBase("a^n") # advective term

G_fn=IndexedBase("G^n")       # pressure term
G_fnp1=IndexedBase("G^{n+1}") # 

# NUMERICAL
g=Symbol('g') # gravity
dt=Symbol("\Delta t")
theta=Symbol(r"\theta")

In [382]:
# Kleptsova, eq 5, simplified for a single layer,
# and ignoring the erroneous trailing "= 0"
eq5a_orig=Eq( A_c[c] * eta_np1[c],
             A_c[c] * eta_n[c]  
             - theta*dt * Sum( sfc[f,c]*lf[f] * h_fn[f] * u_fnp1[f],
                           (f,1,N_f))
             - (1-theta)*dt * Sum( sfc[f,c]*lf[f]*h_fn[f] * u_fn[f],
                               (f,1,N_f)) )
eq5a_orig

Eq(A[c]*\eta^{n+1}[c], -\Delta t*\theta*Sum(hf^n[f]*l[f]*s[f, c]*u^{n+1}[f], (f, 1, N_f)) - \Delta t*(1 - \theta)*Sum(hf^n[f]*l[f]*s[f, c]*u^n[f], (f, 1, N_f)) + A[c]*\eta^n[c])

In [383]:
# Kleptsova, eq 5, rewritten with S,F instead of sfc, and combine the
# summations for clarity
eq5a=Eq( A_c[c] * eta_np1[c],
         A_c[c] * eta_n[c]  
         - Sum(    dt * S[c,fi] *lf[F[c,fi]] * h_fn[F[c,fi]] 
               * ( theta*u_fnp1[F[c,fi]] + (1-theta)*u_fn[F[c,fi]]),
                           (fi,0,Nf_c[c]-1)) )
eq5a

Eq(A[c]*\eta^{n+1}[c], A[c]*\eta^n[c] - Sum(\Delta t*(\theta*u^{n+1}[F[c, fi]] + (1 - \theta)*u^n[F[c, fi]])*hf^n[F[c, fi]]*l[F[c, fi]]*S[c, fi], (fi, 0, Nf[c] - 1)))

In [384]:
# Kleptsova eq 5, omitting Coriolis, and simplified for single layer.
# Use fw=wildcard f to simplify substitution below
eq5b=Eq( u_fnp1[fw],
        u_fn[fw] + dt*a_fn[fw]
        -g*dt*(theta*G_fnp1[fw] + (1-theta)*G_fn[fw]) )
eq5b

Eq(u^{n+1}[f_], -\Delta t*g*(\theta*G^{n+1}[f_] + (1 - \theta)*G^n[f_]) + \Delta t*a^n[f_] + u^n[f_])

In [403]:
# Definition of h_fn
# rather than continue with their sfc notation, use the
# more direct c1(f)
eq8a=Eq( h_fn[fw],
        Piecewise( (h_cn[c1[fw]],  u_fn[fw]>0),
                   (h_cn[c2[fw]],  u_fn[fw]<0),
                   (Max( eta_n[c1[fw]], eta_n[c2[fw]])
                    -
                    Max( b[c1[fw]], b[c2[fw]]),
                    True) ) )
display(eq8a)
eq8b=Eq( h_cn[cw],
         eta_n[cw] - b[cw])
display(eq8b)

eq8=eq8a.replace(eq8b.lhs,eq8b.rhs)
eq8                

Eq(hf^n[f_], Piecewise((hc^n[c1[f_]], u^n[f_] > 0), (hc^n[c2[f_]], u^n[f_] < 0), (Max(\eta^n[c1[f_]], \eta^n[c2[f_]]) - Max(b[c1[f_]], b[c2[f_]]), True)))

Eq(hc^n[c_], \eta^n[c_] - b[c_])

Eq(hf^n[f_], Piecewise((\eta^n[c1[f_]] - b[c1[f_]], u^n[f_] > 0), (\eta^n[c2[f_]] - b[c2[f_]], u^n[f_] < 0), (Max(\eta^n[c1[f_]], \eta^n[c2[f_]]) - Max(b[c1[f_]], b[c2[f_]]), True)))

In [386]:
# The solution procedure
# eq5a give the freesurface update.  But it has the implicit velocity
# in it.  So we substitute, but use replace because the indexes 
# defeat subs().
eq_comb1=eq5a.replace(eq5b.lhs,eq5b.rhs)
eq_comb1

Eq(A[c]*\eta^{n+1}[c], A[c]*\eta^n[c] - Sum(\Delta t*(\theta*(-\Delta t*g*(\theta*G^{n+1}[F[c, fi]] + (1 - \theta)*G^n[F[c, fi]]) + \Delta t*a^n[F[c, fi]] + u^n[F[c, fi]]) + (1 - \theta)*u^n[F[c, fi]])*hf^n[F[c, fi]]*l[F[c, fi]]*S[c, fi], (fi, 0, Nf[c] - 1)))

In [387]:
# Define G operator
# in 3D this would also include baroclinic, which would
# be explicit
eqG=Eq( G_fn[fw],
        (eta_n[c2[fw]]-eta_n[c1[fw]])/lf[fw])
eqGn=eqG
eqGnp1=eqG.subs(eta_n,eta_np1).subs(G_fn,G_fnp1)
display(eqGn)
eqGnp1

Eq(G^n[f_], (-\eta^n[c1[f_]] + \eta^n[c2[f_]])/l[f_])

Eq(G^{n+1}[f_], (-\eta^{n+1}[c1[f_]] + \eta^{n+1}[c2[f_]])/l[f_])

In [388]:
eq_comb2=eq_comb1.replace(eqGn.lhs,eqGn.rhs)
eq_comb2=eq_comb2.replace(eqGnp1.lhs,eqGnp1.rhs)
eq_comb2=eq_comb2.replace(a_fn[fw],0) # while testing, drop advection
eq_comb2

Eq(A[c]*\eta^{n+1}[c], A[c]*\eta^n[c] - Sum(\Delta t*(\theta*(-\Delta t*g*(\theta*(-\eta^{n+1}[c1[F[c, fi]]] + \eta^{n+1}[c2[F[c, fi]]])/l[F[c, fi]] + (1 - \theta)*(-\eta^n[c1[F[c, fi]]] + \eta^n[c2[F[c, fi]]])/l[F[c, fi]]) + u^n[F[c, fi]]) + (1 - \theta)*u^n[F[c, fi]])*hf^n[F[c, fi]]*l[F[c, fi]]*S[c, fi], (fi, 0, Nf[c] - 1)))

In [389]:
# How does this work when evaluating?
# Could iterate over c, substituting everything, to get a series of equations, and then
# just ask sympy to solve.

In [482]:
# Now make some of those concrete:

# 3 cells in a row.
# two faces.

# Inputs:
inp={theta: 0.55,
     dt: 1.0,
     g: 9.8, # gravity
     N_f: 2, # global number of faces
     N_c: 3, # global number of cells
     Nf_max: 2, # maximum number of sides for a cell
     Nf_c:Matrix([1,2,1]), # number of sides for cell c
     #F:Matrix( [[0,-1], # F[c,i] the i'th face adjacent to cell c
     #           [0,1],
     #          [1,-1]]),  
     #S:Matrix( [[1,0], # S[c,i] sign of the normal of F[c,i] w.r.t. c's outward normal
     #           [-1,1],
     #           [-1,0]] ), 
     c1:Matrix( [0,1]), # 'upstream' cell for edge f
     c2:Matrix( [1,2]), # 'downstream' cell for edge f
     A_c:Matrix( [1.0,1.0,1.0]), # cell area
     lf:Matrix( [1,1] ), # length of face f
     b:Matrix( [0,0,0] ), # cell bed elevation
    }

Freal=Matrix( [[0,-1], # F[c,i] the i'th face adjacent to cell c
                [0,1],
                [1,-1]])

Sreal=Matrix( [[1,0], # S[c,i] sign of the normal of F[c,i] w.r.t. c's outward normal
               [-1,1],
               [-1,0]] )

state={
     # h_fn is computed # edge depth
     # h_cn is computed # water column depth
     eta_n: Matrix([1.1,1,1]),
     # eta_np1 is computed 

     u_fn: Matrix([0.0,0.0]) # edge normal velocity
     # u_fnp1 is computed 
     # a_fn is computed # advective term
}

def sub_all(expr):
    for k in inp:
        expr=expr.replace(k,inp[k])
    # eventually the parts that depend on state might be split off
    for k in state:
        expr=expr.replace(k,state[k])
    return expr

In [483]:
# Precalculate h_fn

# Not sure why in this case subs() is required, and replace()
# gives an error.
eq8_0=eq8.rhs.replace(c1,inp[c1]).replace(c2,inp[c2]).subs(eta_n,state[eta_n]).subs(u_fn,state[u_fn])
eq8_1=eq8_0.subs(b,inp[b])

h_fn_real=Matrix( [ eq8_1.subs(fw,i) for i in range(inp[N_f])] )
state[h_fn]=h_fn_real
h_fn_real

Matrix([
[1.1],
[  1]])

In [484]:
eq_real=sub_all(eq_comb2)
Fw=sub_all(F)
Sw=sub_all(S)
eq_real=eq_real.replace(Fw,Freal).replace(Sw,Sreal)

In [485]:
cell_eqs=[  eq_real.subs(c,i).doit()
          for i in range(inp[N_c]) ]
cell_eqs=[ e.rhs-e.lhs for e in cell_eqs]
unknowns=[eta_np1[i] for i in range(inp[N_c])]
for e in cell_eqs:
    display(e)

-1.0326095*\eta^{n+1}[0] + 0.0326095*\eta^{n+1}[1] + 1.09733195

0.0326095*\eta^{n+1}[0] - 1.0622545*\eta^{n+1}[1] + 0.029645*\eta^{n+1}[2] + 1.00266805

0.029645*\eta^{n+1}[1] - 1.029645*\eta^{n+1}[2] + 1.0

In [495]:
eta_set=linsolve(cell_eqs,unknowns)
eta_np1_real=Matrix(list(eta_set)[0])
state[eta_np1]=eta_np1_real
eta_np1_real


Matrix([
[1.09442923578866],
[ 1.0054148623289],
[1.00015590188244]])

In [503]:
# And solve for velocity:
eq5c=eq5b.replace(eqGn.lhs,eqGn.rhs).replace(eqGnp1.lhs,eqGnp1.rhs).replace(a_fn[fw],0)
eq5d=sub_all(eq5c)

u_fnp1_real=Matrix( [eq5d.rhs.subs(fw,i) for i in range(inp[N_f])] )
state[u_fnp1]=u_fnp1_real
u_fnp1_real

Matrix([
[ 0.0920787472948126],
[0.00283457968064464]])

Advection
---


In [321]:
delta=IndexedBase(r'\delta') # ==|sfc|: delta[j,c]=1 if edge j is part of cell c
j=Symbol('j')
alpha=IndexedBase(r'\alpha') # weighting for cell c on edge j
hbar=IndexedBase(r'\overline{h}')
ufstar=IndexedBase(r'\mathbf{u^*}')
nf=IndexedBase(r'\mathbf{n}')
N_c=Symbol('N_c')

In [322]:
# the 'f' in a_fn is generic.  it will get rendered as a_j^n
Eq( a_fn[j],
   Sum(delta[j,c] * alpha[j,c] * 
              Sum(sfc[f,c]*h_fn[f]*lf[f]/(A_c[c]*hbar[j]) * u_fn[f]*(ufstar[f]*nf[j]-u_fn[j]),
                  (f,1,N_f)),
              (c,  1, N_c) ))

Eq(a^n[j], Sum(\alpha[j, c]*\delta[j, c]*Sum((\mathbf{n}[j]*\mathbf{u^*}[f] - u^n[j])*hf^n[f]*l[f]*s[f, c]*u^n[f]/(A[c]*\overline{h}[j]), (f, 1, N_f)), (c, 1, N_c)))

In [323]:
# it's worth seeing if I can solve the rest, and if
# it proves worthwhile then come back to define u*, hbar, n etc.